In [1]:
import numpy as np
import os
import glob
import requests
import pandas as pd
from datetime import datetime, timedelta

from matplotlib import pyplot as plt

from sklearn.ensemble import HistGradientBoostingRegressor

from rich.progress import track

import json
import statsmodels.formula.api as smf

### Dataloader

In [3]:
print("--- 1. Loading CSV data from OSF file ---")
DATA_DIR = 'hrl_load_metered_2016-2025/'
search_path = os.path.join(DATA_DIR, '**', '*.csv')
csv_files = sorted(glob.glob(search_path, recursive=True))
if not csv_files:
    print(f"Error: No CSV files found in {DATA_DIR}.")
    dataframes = {}
else:
    print(f"Found {len(csv_files)} CSV files. Loading each one...")
    dataframes = {}
    all_dates = []  
    for file_path in csv_files:
        file_name = os.path.basename(file_path)
        try:
            df = pd.read_csv(file_path)
            date_col = df.columns[0]
            parsed_dates = pd.to_datetime(df[date_col], errors='coerce')
            all_dates.append(parsed_dates)
            dataframes[file_name] = df
            print(f"  - Successfully loaded and parsed dates from {file_name}")
        except Exception as e:
            print(f"Could not load {file_name}: {e}")

    print(f"\nSuccessfully loaded {len(dataframes)} DataFrames.")

#  --weather
if not dataframes:
    print("\nNo dataframes loaded, skipping weather fetch.")
else:
    print("\n--- 2. Fetching Weather Data for CSV Date Range ---")
    all_dates_combined = pd.concat(all_dates).dropna()
    min_date = all_dates_combined.min()
    max_date = all_dates_combined.max()
    
    # Format dates for the API (YYYY-MM-DD)
    start_str = min_date.strftime('%Y-%m-%d')
    end_str = max_date.strftime('%Y-%m-%d')
    
    print(f"Date range found in CSVs: {start_str} to {end_str}")
    print("Fetching corresponding weather data from Open-Meteo...")

    API_URL = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": 39.95,   # Philadelphia
        "longitude": -75.16, # Philadelphia
        "start_date": start_str,
        "end_date": end_str,
        "hourly": "temperature_2m"
    }

    try:
        response = requests.get(API_URL, params=params)
        response.raise_for_status() 
        weather_data = response.json()
        print("Successfully fetched and parsed weather data.")
        print("\n--- 3. Processing and Merging Weather Data ---")
        hourly_data = weather_data.get('hourly', {})
        if not hourly_data:
            print("Error: No 'hourly' data found in API response.")
        else:
            # Create a clean, time-indexed DataFrame for the weather
            weather_df = pd.DataFrame(hourly_data)
            weather_df['time'] = pd.to_datetime(weather_df['time'])
            weather_df = weather_df.set_index('time')
            print("Created weather DataFrame. Head:")
            print(weather_df.head())

            # Now, let's merge this weather data back into your original CSVs
            # We'll just show an example using the first DataFrame
            
            first_df_key = list(dataframes.keys())[0]
            print(f"\nExample merge with first file: '{first_df_key}'")
            
            original_df = dataframes[first_df_key]
            date_col = original_df.columns[0]
            
            # Parse dates and set as index (this time on the original df)
            original_df[date_col] = pd.to_datetime(original_df[date_col])
            original_df = original_df.set_index(date_col)
            
            # Merge! This joins the temperature to the matching timestamp.
            # 'how=left' keeps all your original data.
            merged_df = original_df.merge(weather_df, left_index=True, right_index=True, how='left')
            
            print("Merged DataFrame (head):")
            print(merged_df.head())

    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
    except json.JSONDecodeError:
        print("Error: Could not decode JSON response from weather API.")
    except Exception as e:
        print(f"An error occurred during weather processing: {e}")
# --- 3. Processing and Merging Weather Data (for ALL CSVs) ---

hourly_data = weather_data.get('hourly', {})
if not hourly_data:
    print("Error: No 'hourly' data found in API response.")
else:
    # 3a) Clean, index the weather
    weather_df = pd.DataFrame(hourly_data)
    weather_df['time'] = pd.to_datetime(weather_df['time'], errors='coerce')
    weather_df = weather_df.dropna(subset=['time']).set_index('time').sort_index()
    print("Created weather DataFrame. Head:")
    print(weather_df.head())

    # 3b) Merge weather into EVERY loaded CSV
    merged_dataframes = {}
    print("\nMerging weather with all CSVs...")
    for fname, original_df in dataframes.items():
        df_i = original_df.copy()
        date_col = df_i.columns[0]                      # assume first column is the timestamp
        df_i[date_col] = pd.to_datetime(df_i[date_col], errors='coerce')
        df_i = df_i.dropna(subset=[date_col]).set_index(date_col).sort_index()

        merged_i = df_i.merge(weather_df, left_index=True, right_index=True, how='left')
        merged_i["source_file"] = fname                 # keep provenance if you want
        merged_dataframes[fname] = merged_i

        missing = int(merged_i["temperature_2m"].isna().sum())
        print(f"  - merged {fname}: {len(merged_i):,} rows (missing temperature_2m: {missing:,})")

    print(f"\nMerged weather into {len(merged_dataframes)} files.")

    # 3c) (Optional) Concatenate into one big DataFrame
    merged_df = (
        pd.concat(merged_dataframes.values(), axis=0, ignore_index=False)
          .rename_axis("timestamp")                    # index name for the merged time column
          .reset_index()
    )
    print("\nCombined merged_df shape:", merged_df.shape)
    print(merged_df.head())
print("\n--- Python script execution finished. ---")

--- 1. Loading CSV data from OSF file ---
Found 10 CSV files. Loading each one...
  - Successfully loaded and parsed dates from hrl_load_metered_2016.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2017.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2018.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2019.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2020.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2021.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2022.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2023.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2024.csv
  - Successfully loaded and parsed dates from hrl_load_metered_2025.csv

Successfully loaded 10 DataFrames.

--- 2. Fetching Weather Data for CSV Date Range ---
Date range found in CSVs: 2016-01-01 to 2025-11-17
Fetching corresponding weather data from Open-Meteo...
Suc

In [115]:
load_area = [
    "AECO",
    "AEPAPT",
    "AEPIMP",
    "AEPKPT",
    "AEPOPT",
    "AP",
    "BC",
    "CE",
    "DAY",
    "DEOK",
    "DOM",
    "DPLCO",
    "DUQ",
    "EASTON",
    "EKPC",
    "JC",
    "ME",
    "OE",
    "OVEC",
    "PAPWR",
    "PE",
    "PEPCO",
    "PLCO",
    "PN",
    "PS",
    "RECO",
    "SMECO",
    "UGI",
    "VMEU"
]

## Task1

In [25]:
def rmse(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def mae(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    return float(np.mean(np.abs(y_true - y_pred)))

def mape(y_true, y_pred):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0)


In [27]:
df = merged_df.copy()
if 'load_area' in globals():
    df = df[df["load_area"].isin(load_area)].copy()

# timestamps (EPT wall-clock)
df["ts"] = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")
df = df.dropna(subset=["ts"]).sort_values(["load_area", "ts"]).reset_index(drop=True)
df["year"] = df["ts"].dt.year
df["hour"] = df["ts"].dt.hour
df["doy"]  = df["ts"].dt.dayofyear  

def add_last_year_feats(frame: pd.DataFrame, n_years: int = 4) -> pd.DataFrame:
    out = frame
    base = (frame[["load_area", "ts", "mw"]]
            .groupby(["load_area", "ts"], as_index=False)["mw"].mean())  # collapse any DST dupes
    for k in range(1, n_years + 1):
        prev = base.rename(columns={"mw": f"mw_ly{k}"}).copy()
        prev["ts"] = prev["ts"] + pd.DateOffset(years=k)  # align prior-year load to current timestamp
        out = out.merge(prev, on=["load_area", "ts"], how="left")
    return out

df = add_last_year_feats(df, n_years=4)


# -------------------------
# 3) Create lag features **by exact timestamp lookup** (not shift),
#    so we can sanitize training rows that would reference holdout.
# -------------------------
def add_lag_by_lookup(frame: pd.DataFrame, hours: int, out_col: str) -> pd.DataFrame:
    key = frame[["load_area", "ts", "mw"]].copy()
    key = key.rename(columns={"mw": out_col})
    key["ts"] = key["ts"] + pd.Timedelta(hours=hours)  # align t-hours -> t
    return frame.merge(key, on=["load_area", "ts"], how="left")

for h, c in [(24, "mw_lag24"), (48, "mw_lag48"), (168, "mw_lag168")]:
    df = add_lag_by_lookup(df, hours=h, out_col=c)
# Define the 10‑day holdout window
HOLD_LO = pd.Timestamp("2025-10-22 00:00:00")
HOLD_HI = pd.Timestamp("2025-11-01 00:00:00")  # exclusive
is_holdout = (df["ts"].ge(HOLD_LO) & df["ts"].lt(HOLD_HI) & df["year"].eq(2025))

# Split (no braces, no dict keys, just a boolean Series aligned to df.index)
valid_df = df.loc[is_holdout].copy()
train_df = df.loc[~is_holdout].copy()

# -------------------------
# 4) SANITIZE TRAIN: any lag whose reference time falls inside holdout is set to NaN.
#    (Prevents leakage of validation actuals into training features.)
# -------------------------
for lag_h, lag_col in [(24, "mw_lag24"), (48, "mw_lag48"), (168, "mw_lag168")]:
    ref_ts = train_df["ts"] - pd.Timedelta(hours=lag_h)
    leak_mask = (ref_ts >= HOLD_LO) & (ref_ts < HOLD_HI)
    train_df.loc[leak_mask, lag_col] = np.nan

# We require all lag features + mw_ly1..mw_ly4 to exist for training
feat_cols = ["mw_lag24", "mw_lag48", "mw_lag168"]
train_df = train_df.dropna(subset=feat_cols + ["mw"]).copy()

print(f"Train rows after leakage sanitization: {len(train_df):,}")

# -------------------------
# 5) Fit boosting model (no CV; uses internal early stopping on TRAIN only)
# -------------------------
X_train = train_df[feat_cols].to_numpy()
y_train = train_df["mw"].to_numpy()

hgb = HistGradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.05,
    max_iter=2000,
    early_stopping=False,     # TRAIN-only internal holdout
    n_iter_no_change=200,
    validation_fraction=0.1, # fraction of TRAIN
    max_bins=255,
    min_samples_leaf=50,
    l2_regularization=1.0,
    max_depth=None,
    random_state=42
)
hgb.fit(X_train, y_train)

# -------------------------
# 6) PREDICT VALIDATION **without using validation actuals**:
#    Roll forward per (area, ts): if a required lag points inside holdout,
#    use our prior PREDICTION for that lag time (never the actual).
# -------------------------
# Known dictionary: all training actuals only
known = {(r.load_area, r.ts): r.mw for r in train_df[["load_area","ts","mw"]].itertuples(index=False)}

# Prepare container for predictions
valid_df = valid_df.sort_values(["load_area", "ts"]).copy()
valid_df["pred"] = np.nan

def row_feat_vector(row, known_map):
    area, ts = row["load_area"], row["ts"]
    l24  = known_map.get((area, ts - pd.Timedelta(hours=24)),  np.nan)
    l48  = known_map.get((area, ts - pd.Timedelta(hours=48)),  np.nan)
    l168 = known_map.get((area, ts - pd.Timedelta(hours=168)), np.nan)
    return np.array([[l24, l48, l168]])

# Roll forward within each area
for area, g in valid_df.groupby("load_area", sort=False):
    idxs = g.sort_values("ts").index
    for idx in idxs:
        feats = row_feat_vector(valid_df.loc[idx], known)
        yhat  = float(hgb.predict(feats)[0])
        valid_df.at[idx, "pred"] = yhat
        # update knowledge so downstream lags can use our prediction
        known[(area, valid_df.at[idx, "ts"])] = yhat

# -------------------------
# 7) Metrics on the 10-day validation window
# -------------------------
y_valid = valid_df["mw"].to_numpy()
y_hat   = valid_df["pred"].to_numpy()

print("\nValidation (NO validation info used in train or features): 2025-10-22..2025-10-31")
print(f"  RMSE: {rmse(y_valid, y_hat):,.2f} MW")
print(f"  MAE : {mae(y_valid,  y_hat):,.2f} MW")
print(f"  MAPE: {mape(y_valid, y_hat):,.2f}%")


/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_94108/3509989948.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ts"] = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")


Train rows after leakage sanitization: 2,392,351

Validation (NO validation info used in train or features): 2025-10-22..2025-10-31
  RMSE: 233.73 MW
  MAE : 137.63 MW
  MAPE: 5.46%


## Task 2

In [10]:
import numpy as np
import pandas as pd

# -------------------------
# 1) Get training residual scale (NO validation info)
# -------------------------
# Predict on the final training set to compute residuals
X_tr = train_df[["mw_lag24", "mw_lag48", "mw_lag168"]].to_numpy()
y_tr = train_df["mw"].to_numpy()
train_df = train_df.copy()
train_df["pred_tr"] = hgb.predict(X_tr)
train_df["resid"] = train_df["mw"] - train_df["pred_tr"]

# Robust hourly sigma per (area, hour): 1.4826 * MAD
def robust_sigma(s):
    med = np.median(s)
    mad = np.median(np.abs(s - med))
    if np.isnan(mad) or mad == 0:
        # fallback to std or small floor to avoid zero variance
        std = np.std(s)
        return float(std if std > 1e-6 else 1.0)
    return float(1.4826 * mad)

sigma_map = (
    train_df.groupby(["load_area", "hour"], observed=True)["resid"]
            .apply(robust_sigma)
)

# Global hour-only fallback if a (area,hour) combo is missing
sigma_hour_global = (
    train_df.groupby("hour", observed=True)["resid"]
            .apply(robust_sigma)
)

def get_sigma_vec(area, hours):
    """Return a vector of sigma values for the given area and iterable of hours."""
    sig = []
    for h in hours:
        key = (area, int(h))
        if key in sigma_map.index:
            sig.append(float(sigma_map.loc[key]))
        else:
            sig.append(float(sigma_hour_global.loc[int(h)]))
    return np.array(sig, dtype=float)

# -------------------------
# 2) Peak-hour selectors
# -------------------------
def soft_argmax(mu, kernel=(1, 3, 1)):
    """Argmax after small smoothing kernel (aligned with ±1 success)."""
    k = np.array(kernel, dtype=float)
    sm = np.convolve(mu, k, mode="same")
    return int(np.argmax(sm))

def probabilistic_peak(mu, sigma, n_draws=1000, rng=None):
    """
    Monte-Carlo selector maximizing Pr(|ĥ − peak| ≤ 1).
    mu: (24,) mean vector for the day
    sigma: (24,) std vector (from training residuals)
    """
    if rng is None:
        rng = np.random.default_rng(42)
    # draw iid normals; (n_draws, 24)
    Z = rng.standard_normal((n_draws, len(mu)))
    Y = mu[None, :] + sigma[None, :] * Z

    # index of max per draw
    hmax = np.argmax(Y, axis=1)

    # vote for (hmax-1,hmax,hmax+1)
    votes = np.zeros(len(mu), dtype=int)
    for h in hmax:
        votes[h] += 1
        if h - 1 >= 0: votes[h - 1] += 1
        if h + 1 < len(mu): votes[h + 1] += 1
    return int(np.argmax(votes))

# -------------------------
# 3) Build per-day, per-area predictions and evaluate
# -------------------------
# Keep only the last-10-days holdout (valid_df) and ensure we have all 24 hours per day
valid_df = valid_df.copy()
valid_df["date"] = valid_df["ts"].dt.date

rows = []
for (area, day), g in valid_df.groupby(["load_area", "date"], sort=True):
    g = g.sort_values("hour")
    if g["hour"].nunique() < 24:
        # skip partial days (shouldn't occur in this window, but be safe)
        continue

    mu = g["pred"].to_numpy()      # our forecast means for that day
    sig = get_sigma_vec(area, g["hour"].to_numpy())
    y  = g["mw"].to_numpy()

    # true peak hour
    h_true = int(np.argmax(y))

    # selectors
    h_argmax  = int(np.argmax(mu))
    h_soft    = soft_argmax(mu, kernel=(1, 3, 1))
    h_prob    = probabilistic_peak(mu, sig, n_draws=1000)

    rows.append({
        "load_area": area,
        "date": day,
        "peak_true": h_true,
        "peak_argmax": h_argmax,
        "peak_soft": h_soft,
        "peak_prob": h_prob
    })

ph_df = pd.DataFrame(rows).sort_values(["load_area", "date"]).reset_index(drop=True)

def acc_pm1(pred, truth):
    pred = np.asarray(pred, dtype=int)
    truth = np.asarray(truth, dtype=int)
    return float(np.mean(np.abs(pred - truth) <= 1))

# Overall ±1 accuracy
acc_argmax = acc_pm1(ph_df["peak_argmax"], ph_df["peak_true"])
acc_soft   = acc_pm1(ph_df["peak_soft"],   ph_df["peak_true"])
acc_prob   = acc_pm1(ph_df["peak_prob"],   ph_df["peak_true"])

print("\nPeak-hour ±1 accuracy on last 10 days (all 29 areas):")
print(f"  Raw argmax of forecast:  {acc_argmax*100:5.1f}%")
print(f"  Soft-argmax (kernel [1,3,1]): {acc_soft*100:5.1f}%")
print(f"  Probabilistic selector:   {acc_prob*100:5.1f}%")

# (Optional) accuracy per area
per_area = (
    ph_df.groupby("load_area", observed=True)
         .apply(lambda d: pd.Series({
             "acc_argmax": acc_pm1(d["peak_argmax"], d["peak_true"]),
             "acc_soft":   acc_pm1(d["peak_soft"],   d["peak_true"]),
             "acc_prob":   acc_pm1(d["peak_prob"],   d["peak_true"]),
         }))
         .sort_values("acc_prob", ascending=False)
)
print("\nPer-area ±1 accuracy (10-day window):")
print(per_area)

# If you want the chosen peak to submit, pick the strongest selector:
ph_df["peak_submit"] = ph_df["peak_prob"]   # or "peak_soft"



Peak-hour ±1 accuracy on last 10 days (all 29 areas):
  Raw argmax of forecast:   59.1%
  Soft-argmax (kernel [1,3,1]):  59.9%
  Probabilistic selector:    57.1%

Per-area ±1 accuracy (10-day window):
           acc_argmax  acc_soft  acc_prob
load_area                                
AECO         1.000000  1.000000  1.000000
PS           1.000000  1.000000  1.000000
JC           1.000000  1.000000  1.000000
CE           0.888889  0.888889  0.888889
RECO         1.000000  1.000000  0.888889
VMEU         0.888889  0.888889  0.888889
BC           0.777778  0.777778  0.777778
DOM          0.555556  0.777778  0.777778
EASTON       0.777778  0.777778  0.777778
SMECO        0.777778  0.777778  0.666667
DPLCO        0.666667  0.666667  0.666667
PE           0.666667  0.666667  0.666667
PAPWR        0.555556  0.555556  0.555556
AEPKPT       0.666667  0.555556  0.555556
PN           0.555556  0.666667  0.555556
AEPAPT       0.444444  0.555556  0.555556
PLCO         0.444444  0.444444  0.444444


/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_94108/2306562405.py:133: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({


## Task 3

In [12]:
import numpy as np
import pandas as pd

# ------------------------------------------------------------
# 0) Build daily-peak table from your merged_df
# ------------------------------------------------------------
df = merged_df.copy()
if 'load_area' in globals():
    df = df[df["load_area"].isin(load_area)].copy()

df["ts"]   = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")
df         = df.dropna(subset=["ts"]).sort_values(["load_area","ts"]).reset_index(drop=True)
df["date"] = df["ts"].dt.normalize()

# Actual daily peak MW
daily = (df.groupby(["load_area","date"], observed=True)["mw"]
           .max().rename("mw_peak").reset_index())

# ------------------------------------------------------------
# 1) Define corresponding Mon–Sun weeks around Oct 31
# ------------------------------------------------------------
K_PAST = 3  # choose top-3 from 2024 to project onto 2025

CENTER_2024 = pd.Timestamp("2024-10-31")
W24_START   = CENTER_2024.to_period("W-MON").start_time.normalize()
W24_END     = W24_START + pd.Timedelta(days=7)

CENTER_2025 = pd.Timestamp("2025-10-31")
W25_START   = CENTER_2025.to_period("W-MON").start_time.normalize()
W25_END     = W25_START + pd.Timedelta(days=7)

# Slice to those weeks
d24 = daily[(daily["date"] >= W24_START) & (daily["date"] < W24_END)].copy()
d25 = daily[(daily["date"] >= W25_START) & (daily["date"] < W25_END)].copy()

# Keep only areas that exist in BOTH weeks
areas_24 = set(d24["load_area"].unique())
areas_25 = set(d25["load_area"].unique())
common_areas = areas_24 & areas_25
d24 = d24[d24["load_area"].isin(common_areas)].copy()
d25 = d25[d25["load_area"].isin(common_areas)].copy()

# Week positions: 0..6 from the Monday start (more robust than names)
week24_pos = pd.DataFrame({"date": [W24_START + pd.Timedelta(days=i) for i in range(7)], "pos": np.arange(7)})
week25_pos = pd.DataFrame({"date": [W25_START + pd.Timedelta(days=i) for i in range(7)], "pos": np.arange(7)})

d24 = d24.merge(week24_pos, on="date", how="left")
d25 = d25.merge(week25_pos, on="date", how="left")

# ------------------------------------------------------------
# 2) Past-year TOP-3 argmax per area (on 2024 week) → map to 2025 week positions
# ------------------------------------------------------------
topK_2024 = (d24.sort_values(["load_area","mw_peak"], ascending=[True, False])
               .groupby("load_area", as_index=False).head(K_PAST)
               [["load_area","pos"]]
               .groupby("load_area")["pos"].apply(lambda s: sorted(pd.unique(s)))
               .rename("pos_set").reset_index())

# Predict 2025 peak days: those with positions in last year's TOP-3 set
d25 = d25.merge(topK_2024, on="load_area", how="left")
d25["pred_peakday"] = d25.apply(lambda r: int(isinstance(r["pos_set"], list) and (r["pos"] in r["pos_set"])), axis=1)
pred_days = d25[d25["pred_peakday"] == 1][["load_area","date","pred_peakday"]]

# ------------------------------------------------------------
# 3) Ground truth in the 2025 week: top-2 actual days by daily peak
# ------------------------------------------------------------
true_top2 = (d25.sort_values(["load_area","mw_peak"], ascending=[True, False])
               .groupby("load_area", as_index=False).head(2)
               .assign(y_true=1)[["load_area","date","y_true"]])

# ------------------------------------------------------------
# 4) Evaluate with FN=4, FP=1 and report Hit@2
# ------------------------------------------------------------
eval_df = (d25[["load_area","date"]]
           .merge(true_top2, on=["load_area","date"], how="left")
           .merge(pred_days, on=["load_area","date"], how="left")
           .fillna({"y_true":0, "pred_peakday":0})
           .sort_values(["load_area","date"]).reset_index(drop=True))

FN = int(((eval_df["y_true"]==1) & (eval_df["pred_peakday"]==0)).sum())
FP = int(((eval_df["y_true"]==0) & (eval_df["pred_peakday"]==1)).sum())
TP = int(((eval_df["y_true"]==1) & (eval_df["pred_peakday"]==1)).sum())

areas = eval_df["load_area"].nunique()
cost_total = 4*FN + FP
hit_at_2   = TP / max(1, 2*areas)  # two true peak days per area

print(f"[2024→2025 week | TOP-{K_PAST} baseline]  Areas={areas}")
print(f"  Total cost = {cost_total}   |   FN = {FN}   FP = {FP}   |   Hit@2 = {hit_at_2:.2%}")

# Per-area breakdown
def _area_stats(d):
    y = d["y_true"].to_numpy(int); p = d["pred_peakday"].to_numpy(int)
    tp = int(((y==1) & (p==1)).sum())
    fp = int(((y==0) & (p==1)).sum())
    fn = int(((y==1) & (p==0)).sum())
    return pd.Series({"Hit@2": tp, "FP": fp, "FN": fn, "Cost": 4*fn + fp})

per_area = (eval_df.groupby("load_area", observed=True)
            .apply(_area_stats).sort_values("Cost", ascending=True))
print("\nPer-area (Hit@2, FP, FN, Cost):")
print(per_area)


/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_94108/2442827528.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ts"]   = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")


[2024→2025 week | TOP-3 baseline]  Areas=28
  Total cost = 101   |   FN = 20   FP = 21   |   Hit@2 = 64.29%

Per-area (Hit@2, FP, FN, Cost):
           Hit@2  FP  FN  Cost
load_area                     
AEPIMP         2   0   0     0
AEPOPT         2   0   0     0
PLCO           2   0   0     0
CE             2   0   0     0
DUQ            2   0   0     0
OVEC           2   0   0     0
ME             2   0   0     0
DPLCO          2   1   0     1
OE             2   1   0     1
EASTON         1   1   1     5
RECO           1   1   1     5
PS             1   1   1     5
PN             1   1   1     5
PEPCO          1   1   1     5
PE             1   1   1     5
PAPWR          1   1   1     5
VMEU           1   1   1     5
EKPC           1   1   1     5
SMECO          1   1   1     5
DOM            1   1   1     5
DEOK           1   1   1     5
DAY            1   1   1     5
BC             1   1   1     5
AP             1   1   1     5
AEPKPT         1   1   1     5
AEPAPT         1   1  

/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_94108/2442827528.py:100: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_area_stats).sort_values("Cost", ascending=True))


## Aggregation

In [5]:
df = merged_df.copy()
if 'load_area' in globals():
    df = df[df["load_area"].isin(load_area)].copy()

# timestamps (EPT wall-clock)
df["ts"] = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")
df = df.dropna(subset=["ts"]).sort_values(["load_area", "ts"]).reset_index(drop=True)
df["year"] = df["ts"].dt.year
df["hour"] = df["ts"].dt.hour
df["doy"]  = df["ts"].dt.dayofyear  

def add_last_year_feats(frame: pd.DataFrame, n_years: int = 4) -> pd.DataFrame:
    out = frame
    base = (frame[["load_area", "ts", "mw"]]
            .groupby(["load_area", "ts"], as_index=False)["mw"].mean())  # collapse any DST dupes
    for k in range(1, n_years + 1):
        prev = base.rename(columns={"mw": f"mw_ly{k}"}).copy()
        prev["ts"] = prev["ts"] + pd.DateOffset(years=k)  # align prior-year load to current timestamp
        out = out.merge(prev, on=["load_area", "ts"], how="left")
    return out

df = add_last_year_feats(df, n_years=4)


# -------------------------
# 3) Create lag features **by exact timestamp lookup** (not shift),
#    so we can sanitize training rows that would reference holdout.
# -------------------------
def add_lag_by_lookup(frame: pd.DataFrame, hours: int, out_col: str) -> pd.DataFrame:
    key = frame[["load_area", "ts", "mw"]].copy()
    key = key.rename(columns={"mw": out_col})
    key["ts"] = key["ts"] + pd.Timedelta(hours=hours)  # align t-hours -> t
    return frame.merge(key, on=["load_area", "ts"], how="left")

for h, c in [(24, "mw_lag24"), (48, "mw_lag48"), (168, "mw_lag168")]:
    df = add_lag_by_lookup(df, hours=h, out_col=c)
# Define the 10‑day holdout window
HOLD_LO = pd.Timestamp("2025-10-22 00:00:00")
HOLD_HI = pd.Timestamp("2025-11-01 00:00:00")  # exclusive
is_holdout = (df["ts"].ge(HOLD_LO) & df["ts"].lt(HOLD_HI) & df["year"].eq(2025))

# Split (no braces, no dict keys, just a boolean Series aligned to df.index)
valid_df = df.loc[is_holdout].copy()
train_df = df.loc[~is_holdout].copy()

# -------------------------
# 4) SANITIZE TRAIN: any lag whose reference time falls inside holdout is set to NaN.
#    (Prevents leakage of validation actuals into training features.)
# -------------------------
for lag_h, lag_col in [(24, "mw_lag24"), (48, "mw_lag48"), (168, "mw_lag168")]:
    ref_ts = train_df["ts"] - pd.Timedelta(hours=lag_h)
    leak_mask = (ref_ts >= HOLD_LO) & (ref_ts < HOLD_HI)
    train_df.loc[leak_mask, lag_col] = np.nan

# We require all lag features + mw_ly1..mw_ly4 to exist for training
feat_cols = ["mw_lag24", "mw_lag48", "mw_lag168"]
train_df = train_df.dropna(subset=feat_cols + ["mw"]).copy()

print(f"Train rows after leakage sanitization: {len(train_df):,}")

# -------------------------
# 5) Fit boosting model (no CV; uses internal early stopping on TRAIN only)
# -------------------------
X_train = train_df[feat_cols].to_numpy()
y_train = train_df["mw"].to_numpy()

hgb = HistGradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.05,
    max_iter=2000,
    early_stopping=False,     # TRAIN-only internal holdout
    n_iter_no_change=200,
    validation_fraction=0.1, # fraction of TRAIN
    max_bins=255,
    min_samples_leaf=50,
    l2_regularization=1.0,
    max_depth=None,
    random_state=42
)
hgb.fit(X_train, y_train)

# -------------------------
# 6) PREDICT VALIDATION **without using validation actuals**:
#    Roll forward per (area, ts): if a required lag points inside holdout,
#    use our prior PREDICTION for that lag time (never the actual).
# -------------------------
# Known dictionary: all training actuals only
known = {(r.load_area, r.ts): r.mw for r in train_df[["load_area","ts","mw"]].itertuples(index=False)}

# Prepare container for predictions
valid_df = valid_df.sort_values(["load_area", "ts"]).copy()
valid_df["pred"] = np.nan

def row_feat_vector(row, known_map):
    area, ts = row["load_area"], row["ts"]
    l24  = known_map.get((area, ts - pd.Timedelta(hours=24)),  np.nan)
    l48  = known_map.get((area, ts - pd.Timedelta(hours=48)),  np.nan)
    l168 = known_map.get((area, ts - pd.Timedelta(hours=168)), np.nan)
    return np.array([[l24, l48, l168]])

# Roll forward within each area
for area, g in valid_df.groupby("load_area", sort=False):
    idxs = g.sort_values("ts").index
    for idx in idxs:
        feats = row_feat_vector(valid_df.loc[idx], known)
        yhat  = float(hgb.predict(feats)[0])
        valid_df.at[idx, "pred"] = yhat
        # update knowledge so downstream lags can use our prediction
        known[(area, valid_df.at[idx, "ts"])] = yhat

# -------------------------
# 7) Metrics on the 10-day validation window
# -------------------------
y_valid = valid_df["mw"].to_numpy()
y_hat   = valid_df["pred"].to_numpy()

print("\nValidation (NO validation info used in train or features): 2025-10-22..2025-10-31")
print(f"  RMSE: {rmse(y_valid, y_hat):,.2f} MW")
print(f"  MAE : {mae(y_valid,  y_hat):,.2f} MW")
print(f"  MAPE: {mape(y_valid, y_hat):,.2f}%")


/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_3207/3908537845.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ts"] = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")


Train rows after leakage sanitization: 2,364,657


KeyboardInterrupt: 

In [7]:

load_area = [
    "AECO",
    "AEPAPT",
    "AEPIMP",
    "AEPKPT",
    "AEPOPT",
    "AP",
    "BC",
    "CE",
    "DAY",
    "DEOK",
    "DOM",
    "DPLCO",
    "DUQ",
    "EASTON",
    "EKPC",
    "JC",
    "ME",
    "OE",
    "OVEC",
    "PAPWR",
    "PE",
    "PEPCO",
    "PLCO",
    "PN",
    "PS",
    "RECO",
    "SMECO",
    "UGI",
    "VMEU"
]
# -------------------------------
# 0) Configuration
# -------------------------------
# Train on everything before 2025-11-17 00:00 (i.e., up to and including Nov 16)
TRAIN_END = pd.Timestamp("2025-11-17 00:00:00")  # exclusive
# Roll-forward (consecutive) prediction days (inclusive)
ROLL_START_DAY = pd.Timestamp("2025-11-17").normalize()
ROLL_END_DAY   = pd.Timestamp("2025-11-20").normalize()   # last day you asked to predict

FEATS = ["mw_lag24", "mw_lag48", "mw_lag168"]

# -------------------------------
# 1) Training set (strictly before TRAIN_END)
# -------------------------------

df = merged_df.copy()
df["ts"] = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")
df = df.dropna(subset=["ts"]).sort_values(["load_area", "ts"]).reset_index(drop=True)
df["year"] = df["ts"].dt.year
df["hour"] = df["ts"].dt.hour
df["doy"]  = df["ts"].dt.dayofyear 
df = df[df["load_area"].isin(load_area)].copy()
def add_lag_by_lookup(frame: pd.DataFrame, hours: int, out_col: str) -> pd.DataFrame:
    key = frame[["load_area", "ts", "mw"]].copy()
    key = key.rename(columns={"mw": out_col})
    key["ts"] = key["ts"] + pd.Timedelta(hours=hours)  # align t-hours -> t
    return frame.merge(key, on=["load_area", "ts"], how="left")

for h, c in [(24, "mw_lag24"), (48, "mw_lag48"), (168, "mw_lag168")]:
    df = add_lag_by_lookup(df, hours=h, out_col=c)


train_cut = df.loc[df["ts"] < TRAIN_END].dropna(subset=FEATS + ["mw"]).copy()
X_tr = train_cut[FEATS].to_numpy()
y_tr = train_cut["mw"].to_numpy()

hgb_roll = HistGradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.05,
    max_iter=2000,
    early_stopping=True,
    n_iter_no_change=100,
    validation_fraction=0.10,
    max_bins=255,
    min_samples_leaf=50,
    l2_regularization=1.0,
    random_state=42,
    verbose=0
)
print(f"[Train] rows: {len(train_cut):,}  features: {FEATS}")
hgb_roll.fit(X_tr, y_tr)

# -------------------------------
# 2) Roll-forward across Nov 17 → 18 → 19 → 20
#    Use predictions to populate lag features when needed.
# -------------------------------
# Build hourly grid we will traverse sequentially
roll_hours = pd.date_range(
    ROLL_START_DAY,
    ROLL_END_DAY + pd.Timedelta(days=1),
    freq="H",
    inclusive="left"
)

# Areas to run (use your list if provided; otherwise infer)
areas = sorted(df["load_area"].dropna().unique().tolist())

# Known map seeded ONLY with training actuals (no leakage)
known = {(r.load_area, r.ts): r.mw
         for r in train_cut[["load_area","ts","mw"]].itertuples(index=False)}

def feat_vec(area, ts, known_map):
    """Build 1x3 lag vector using ACTUALS from train or prior PREDICTIONS."""
    l24  = known_map.get((area, ts - pd.Timedelta(hours=24)),  np.nan)
    l48  = known_map.get((area, ts - pd.Timedelta(hours=48)),  np.nan)
    l168 = known_map.get((area, ts - pd.Timedelta(hours=168)), np.nan)
    return np.array([[l24, l48, l168]], dtype=float)

rows = []
for area in areas:
    for ts in roll_hours:
        X = feat_vec(area, ts, known)
        yhat = float(hgb_roll.predict(X)[0])   # HGB supports NaN in features
        rows.append((area, ts, ts.hour, yhat))
        # IMPORTANT: immediately make this available as a lag for downstream hours/days
        known[(area, ts)] = yhat

pred_consecutive = (pd.DataFrame(rows, columns=["load_area","ts","hour","pred"])
                      .sort_values(["load_area","ts"])
                      .reset_index(drop=True))

print(f"[Roll-forward] Predicted {len(pred_consecutive):,} rows "
      f"({len(areas)} areas × {len(roll_hours)} hours).")

# -------------------------------
# 3) (Optional) Metrics where actuals exist in df (for those days)
# -------------------------------
eval_slice = df.loc[(df["ts"] >= ROLL_START_DAY) & (df["ts"] < ROLL_END_DAY + pd.Timedelta(days=1)),
                    ["load_area","ts","mw"]]
eval_df = pred_consecutive.merge(eval_slice, on=["load_area","ts"], how="left")

if eval_df["mw"].notna().any():
    def rmse(a,b):
        a,b = np.asarray(a), np.asarray(b)
        return float(np.sqrt(np.mean((a-b)**2)))
    def mae(a,b):
        a,b = np.asarray(a), np.asarray(b)
        return float(np.mean(np.abs(a-b)))
    def mape(a,b):
        a,b = np.asarray(a), np.asarray(b)
        m = a != 0
        return float(np.mean(np.abs((a[m]-b[m])/a[m]))*100.0) if m.any() else np.nan

    print("\n[Metrics on Nov 17–20 where actuals exist]")
    print(f"  RMSE: {rmse(eval_df['mw'].dropna(), eval_df.loc[eval_df['mw'].notna(), 'pred']):,.2f} MW")
    print(f"  MAE : {mae (eval_df['mw'].dropna(), eval_df.loc[eval_df['mw'].notna(), 'pred']):,.2f} MW")
    print(f"  MAPE: {mape(eval_df['mw'].dropna(), eval_df.loc[eval_df['mw'].notna(), 'pred']):,.2f}%")

# -------------------------------
# 4) Convenience slice: predictions for Nov 20 only (24 hours × areas)
# -------------------------------
pred_2025_11_20 = pred_consecutive[(pred_consecutive["ts"] >= pd.Timestamp("2025-11-20")) &
                                   (pred_consecutive["ts"] <  pd.Timestamp("2025-11-21"))] \
                                  .copy() \
                                  .sort_values(["load_area","ts"])
print(f"\n[Nov 20] rows: {len(pred_2025_11_20):,} (areas × 24 hours)")
print(pred_2025_11_20.head(10).to_string(index=False))


/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_3207/56292226.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ts"] = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")


[Train] rows: 2,244,232  features: ['mw_lag24', 'mw_lag48', 'mw_lag168']


/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_3207/56292226.py:89: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  roll_hours = pd.date_range(


[Roll-forward] Predicted 2,784 rows (29 areas × 96 hours).

[Nov 20] rows: 696 (areas × 24 hours)
load_area                  ts  hour       pred
     AECO 2025-11-20 00:00:00     0 820.970155
     AECO 2025-11-20 01:00:00     1 798.861627
     AECO 2025-11-20 02:00:00     2 786.618239
     AECO 2025-11-20 03:00:00     3 786.618239
     AECO 2025-11-20 04:00:00     4 794.369574
     AECO 2025-11-20 05:00:00     5 826.109342
     AECO 2025-11-20 06:00:00     6 876.250631
     AECO 2025-11-20 07:00:00     7 876.250631
     AECO 2025-11-20 08:00:00     8 820.970155
     AECO 2025-11-20 09:00:00     9 769.758397


In [13]:
df

,timestamp,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified,temperature_2m,source_file,ts,date
0,2017-01-01 05:00:00,1/1/2017 12:00:00 AM,RFC,MIDATL,AE,AECO,1017.985,True,6.4,hrl_load_metered_2017.csv,2017-01-01 00:00:00,2017-01-01
1,2017-01-01 06:00:00,1/1/2017 1:00:00 AM,RFC,MIDATL,AE,AECO,964.157,True,6.3,hrl_load_metered_2017.csv,2017-01-01 01:00:00,2017-01-01
2,2017-01-01 07:00:00,1/1/2017 2:00:00 AM,RFC,MIDATL,AE,AECO,921.292,True,5.9,hrl_load_metered_2017.csv,2017-01-01 02:00:00,2017-01-01
3,2017-01-01 08:00:00,1/1/2017 3:00:00 AM,RFC,MIDATL,AE,AECO,897.977,True,5.8,hrl_load_metered_2017.csv,2017-01-01 03:00:00,2017-01-01
4,2017-01-01 09:00:00,1/1/2017 4:00:00 AM,RFC,MIDATL,AE,AECO,895.976,True,5.6,hrl_load_metered_2017.csv,2017-01-01 04:00:00,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
2252857,2025-11-17 00:00:00,11/16/2025 7:00:00 PM,RFC,MIDATL,AE,VMEU,77.289,False,7.6,hrl_load_metered_2025.csv,2025-11-16 19:00:00,2025-11-16
2252858,2025-11-17 01:00:00,11/16/2025 8:00:00 PM,RFC,MIDATL,AE,VMEU,76.403,False,6.9,hrl_load_metered_2025.csv,2025-11-16 20:00:00,2025-11-16
2252859,2025-11-17 02:00:00,11/16/2025 9:00:00 PM,RFC,MIDATL,AE,VMEU,74.414,False,6.1,hrl_load_metered_2025.csv,2025-11-16 21:00:00,2025-11-16
2252860,2025-11-17 03:00:00,11/16/2025 10:00:00 PM,RFC,MIDATL,AE,VMEU,71.312,False,5.3,hrl_load_metered_2025.csv,2025-11-16 22:00:00,2025-11-16


In [19]:
import numpy as np
import pandas as pd

def robust_sigma(s):
    med = np.median(s)
    mad = np.median(np.abs(s - med))
    if np.isnan(mad) or mad == 0:
        std = np.std(s)
        return float(std if std > 1e-6 else 1.0)
    return float(1.4826 * mad)
train_df = df
X_tr = train_df[["mw_lag24", "mw_lag48", "mw_lag168"]].to_numpy()
y_tr = train_df["mw"].to_numpy()
train_df = train_df.copy()
train_df["pred_tr"] = hgb_roll.predict(X_tr)
train_df["resid"] = train_df["mw"] - train_df["pred_tr"]

sigma_map = (train_df.groupby(["load_area","hour"], observed=True)["resid"]
                        .apply(robust_sigma))
sigma_hour_global = (train_df.groupby("hour", observed=True)["resid"]
                               .apply(robust_sigma))

def get_sigma_vec(area, hours):
    out = []
    for h in hours:
        key = (area, int(h))
        if key in sigma_map.index:
            out.append(float(sigma_map.loc[key]))
        else:
            out.append(float(sigma_hour_global.loc[int(h)]))
    return np.array(out, dtype=float)

# ============================================================
# B) Roll-forward predictions: Nov 17 → 18 → 19 → 20
#    (lags inside this window come from our own predictions)
# ============================================================
ROLL_START_DAY = pd.Timestamp("2025-11-17")
ROLL_END_DAY   = pd.Timestamp("2025-11-20")
FEATS = ["mw_lag24", "mw_lag48", "mw_lag168"]

# Build hourly grid (inclusive of days, exclusive of end midnight)
roll_hours = pd.date_range(ROLL_START_DAY, ROLL_END_DAY + pd.Timedelta(days=1),
                           freq="H", inclusive="left")

areas = sorted(df["load_area"].dropna().unique().tolist())

# Seed known map with ACTUALS strictly before the roll window (no leakage)
known = {(r.load_area, r.ts): r.mw
         for r in df.loc[df["ts"] < ROLL_START_DAY, ["load_area","ts","mw"]]
                   .itertuples(index=False)}

def feat_vec(area, ts, known_map):
    l24  = known_map.get((area, ts - pd.Timedelta(hours=24)),  np.nan)
    l48  = known_map.get((area, ts - pd.Timedelta(hours=48)),  np.nan)
    l168 = known_map.get((area, ts - pd.Timedelta(hours=168)), np.nan)
    return np.array([[l24, l48, l168]], dtype=float)

rows = []
for area in areas:
    for ts in roll_hours:
        X1 = feat_vec(area, ts, known)
        yhat = float(hgb_roll.predict(X1)[0])  # HGB supports NaN in features
        rows.append((area, ts, ts.hour, yhat))
        # Make the new prediction available to downstream lags
        known[(area, ts)] = yhat

pred_consec = (pd.DataFrame(rows, columns=["load_area","ts","hour","pred"])
                 .sort_values(["load_area","ts"])
                 .reset_index(drop=True))

# Extract Nov 20 (24 hours × areas)
pred_2025_11_20 = pred_consec[(pred_consec["ts"] >= pd.Timestamp("2025-11-20")) &
                              (pred_consec["ts"] <  pd.Timestamp("2025-11-21"))] \
                             .copy()

# ============================================================
# C) Peak-hour selectors (same as your Task-2 block)
# ============================================================
def soft_argmax(mu, kernel=(1,3,1)):
    k = np.array(kernel, dtype=float)
    sm = np.convolve(mu, k, mode="same")
    return int(np.argmax(sm))

def probabilistic_peak(mu, sigma, n_draws=2000, rng=None):
    if rng is None:
        rng = np.random.default_rng(42)
    Z = rng.standard_normal((n_draws, len(mu)))
    Y = mu[None, :] + sigma[None, :] * Z
    hmax = np.argmax(Y, axis=1)
    votes = np.zeros(len(mu), dtype=int)
    for h in hmax:
        votes[h] += 1
        if h - 1 >= 0: votes[h - 1] += 1
        if h + 1 < len(mu): votes[h + 1] += 1
    return int(np.argmax(votes))

# ============================================================
# D) Per-area peak hour for Nov 20 + SAVE
# ============================================================
p20 = pred_2025_11_20.copy()
p20["date"] = p20["ts"].dt.normalize()
p20["hour"] = p20["hour"].astype(int)

rows = []
for area, g in p20.groupby("load_area", sort=True):
    gg = g.sort_values("hour")
    # If fewer than 24 hours, still proceed (or `continue` if you prefer strictness)
    mu  = gg["pred"].to_numpy()
    sig = get_sigma_vec(area, gg["hour"].to_numpy())

    idx_arg  = int(np.argmax(mu))
    idx_soft = soft_argmax(mu, kernel=(1,3,1))
    idx_prob = probabilistic_peak(mu, sig, n_draws=2000)

    # Map argmax indices back to *hour labels* present in gg
    peak_arg = int(gg["hour"].iloc[idx_arg])
    peak_soft= int(gg["hour"].iloc[idx_soft])
    peak_prob= int(gg["hour"].iloc[idx_prob])

    rows.append({
        "load_area": area,
        "date":      pd.Timestamp("2025-11-20").date(),
        "peak_hour_argmax": peak_arg,
        "peak_hour_soft":   peak_soft,
        "peak_hour_prob":   peak_prob
    })

peak_hours_2025_11_20 = (pd.DataFrame(rows)
                         .sort_values("load_area")
                         .reset_index(drop=True))
peak_hours_2025_11_20["peak_hour_submit"] = peak_hours_2025_11_20["peak_hour_prob"]
peak_hours_pred = peak_hours_2025_11_20["peak_hour_prob"]

KeyError: "None of [Index(['mw_lag24', 'mw_lag48', 'mw_lag168'], dtype='object')] are in the [columns]"

In [33]:
peak_hours_pred

0     18
1     19
2     19
3      8
4     19
5     18
6     18
7     18
8     18
9     18
10     7
11    18
12    18
13    18
14    19
15    18
16    18
17    18
18    15
19    18
20    18
21    18
22    18
23    17
24    18
25    17
26    19
27    18
28    17
Name: peak_hour_prob, dtype: int64

In [9]:
import numpy as np
import pandas as pd

# ------------------------------
# 0) Daily peak table from merged_df
# ------------------------------
df = merged_df.copy()
if 'load_area' in globals():
    df = df[df["load_area"].isin(load_area)].copy()

df["ts"]   = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")
df         = df.dropna(subset=["ts"]).sort_values(["load_area","ts"]).reset_index(drop=True)
df["date"] = df["ts"].dt.normalize()

# Actual *daily* peak MW per area (for past years; 2025 might be missing/future)
daily = (df.groupby(["load_area","date"], observed=True)["mw"]
           .max().rename("mw_peak").reset_index())

# ------------------------------
# 1) Configuration
# ------------------------------
K_PAST   = 3                                   # use top-3 positions from the corresponding 2024 week
WIN_START = pd.Timestamp("2025-11-20").normalize()
WIN_END   = pd.Timestamp("2025-11-30").normalize()  # exclusive upper bound (→ up to 11/29)

# Helper: Monday-start week for any date
def week_start_monday(dt):
    return pd.Timestamp(dt).to_period("W-MON").start_time.normalize()

# All 2025 dates in the requested window
target_dates_2025 = pd.date_range(WIN_START, WIN_END - pd.Timedelta(days=1), freq="D")

# Unique Monday-start week anchors that intersect the window
week_starts_2025 = sorted({week_start_monday(d) for d in target_dates_2025})

pred_rows = []

for W25_START in week_starts_2025:
    W25_END = W25_START + pd.Timedelta(days=7)
    # Corresponding week in 2024 (same weekday alignment)
    W24_START = W25_START - pd.DateOffset(years=1)
    W24_END   = W24_START + pd.Timedelta(days=7)

    # 2024 week (we *need* this week to derive Top-K positions)
    d24 = daily[(daily["date"] >= W24_START) & (daily["date"] < W24_END)].copy()
    if d24.empty:
        # No 2024 data for this aligned week → skip
        continue

    # Areas observed in the 2024 week
    areas = sorted(d24["load_area"].unique())

    # Build the full 7-day grid for the 2025 aligned week (even if 2025 actuals are missing)
    week25_days = [W25_START + pd.Timedelta(days=i) for i in range(7)]
    grid = pd.MultiIndex.from_product([areas, week25_days], names=["load_area","date"]).to_frame(index=False)

    # Week positions 0..6 for both weeks
    map_pos_24 = pd.DataFrame({
        "date": [W24_START + pd.Timedelta(days=i) for i in range(7)],
        "pos":  np.arange(7)
    })
    map_pos_25 = pd.DataFrame({
        "date": [W25_START + pd.Timedelta(days=i) for i in range(7)],
        "pos":  np.arange(7)
    })

    # Attach positions
    d24  = d24.merge(map_pos_24, on="date", how="left")
    grid = grid.merge(map_pos_25, on="date", how="left")

    # Top-K positions (by daily peak) from 2024 week per area
    topK_2024 = (d24.sort_values(["load_area","mw_peak"], ascending=[True, False])
                   .groupby("load_area", as_index=False).head(K_PAST)
                   [["load_area","pos"]]
                   .groupby("load_area")["pos"]
                   .apply(lambda s: sorted(pd.unique(s)))
                   .rename("pos_set").reset_index())

    # Predict: positions that appeared among last year's Top-K
    grid = grid.merge(topK_2024, on="load_area", how="left")
    grid["pred_peakday"] = grid.apply(
        lambda r: int(isinstance(r["pos_set"], list) and (r["pos"] in r["pos_set"])),
        axis=1
    )

    # (Optional) attach 2025 actual daily peaks if they exist in your data
    grid = grid.merge(daily[["load_area","date","mw_peak"]], on=["load_area","date"], how="left")

    # Keep only the requested window dates
    grid = grid[(grid["date"] >= WIN_START) & (grid["date"] < WIN_END)].copy()

    pred_rows.append(grid[["load_area","date","mw_peak","pred_peakday"]])

# ------------------------------
# 2) Final predictions for 2025-11-20 ... 2025-11-29
# ------------------------------
if pred_rows:
    pred_peakdays_2025 = (pd.concat(pred_rows, ignore_index=True)
                            .sort_values(["load_area","date"])
                            .reset_index(drop=True))
else:
    pred_peakdays_2025 = pd.DataFrame(columns=["load_area","date","mw_peak","pred_peakday"])

print("Predicted peak days (1=yes) between 2025-11-20 and 2025-11-29:")
print(pred_peakdays_2025)

peakday_pred = pred_peakdays_2025[pred_peakdays_2025["date"] == "2025-11-20"]["pred_peakday"]

/var/folders/l0/n_657n7d2xdb6sz6mnqb_5zh0000gr/T/ipykernel_3207/831815034.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ts"]   = pd.to_datetime(df["datetime_beginning_ept"], errors="coerce")


Predicted peak days (1=yes) between 2025-11-20 and 2025-11-29:
    load_area       date  mw_peak  pred_peakday
0        AECO 2025-11-20      NaN             0
1        AECO 2025-11-21      NaN             1
2        AECO 2025-11-22      NaN             1
3        AECO 2025-11-23      NaN             0
4        AECO 2025-11-24      NaN             0
..        ...        ...      ...           ...
285      VMEU 2025-11-25      NaN             1
286      VMEU 2025-11-26      NaN             1
287      VMEU 2025-11-27      NaN             1
288      VMEU 2025-11-28      NaN             0
289      VMEU 2025-11-29      NaN             0

[290 rows x 4 columns]


0      0
10     0
20     1
30     0
40     1
50     0
60     0
70     1
80     1
90     1
100    0
110    0
120    0
130    0
140    0
150    0
160    0
170    1
180    1
190    0
200    0
210    0
220    0
230    1
240    1
250    1
260    0
270    0
280    0
Name: pred_peakday, dtype: int64

In [17]:
# 1) Row count per area
first_two = np.append(pred_2025_11_20["pred"],peak_hours_pred)

df_out = pd.DataFrame(
    {"2025-11-20": np.append(peakday_pred)}
)

df_out.to_csv("pred_2025_11_20.csv", index=False)

NameError: name 'peak_hours_pred' is not defined

In [15]:
print(df_out)

NameError: name 'df_out' is not defined